In [ ]:
# !pip install rdkit
# !pip install torch
# !pip install allennlp

In [1]:
import torch
import pickle as pi
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import tensorflow as tf

In [4]:
import os
os.getcwd()

'd:\\stuff\\code\\aiLab\\help\\modified_version'

In [5]:
clf = pi.load(open('../weights/clf.pkl', 'rb'))

In [ ]:
drug = 'CN1C=NC2=C1C(=O)N(C)C(=O)N2C'

In [ ]:
generated_example = 'NC(=O)CN1CCCC1=O'

In [ ]:
def generate_descriptors(smile):
  
  descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
  get_descriptors = rdMolDescriptors.Properties(descriptor_names)
  
  molecule_object = Chem.MolFromSmiles(smile)
  final_descriptors = np.array(get_descriptors.ComputeProperties(molecule_object)).reshape((-1, 43))

  return final_descriptors


def get_clf_input(coformer_smile, drug_smile=drug):

  drug_descriptors, coformer_descriptors = generate_descriptors(drug_smile), generate_descriptors(coformer_smile)

  final_input = np.concatenate((drug_descriptors, coformer_descriptors), axis=1)

  return final_input


def calculate_clf_error(coformer_smile, desired_clf_output=1, drug_smile=drug, classifier=clf):

  clf_input = get_clf_input(coformer_smile, drug_smile)
  clf_prediction = classifier.predict_proba(clf_input)[:,desired_clf_output]

  error = tf.keras.metrics.binary_crossentropy(desired_clf_output, 
                                               clf_prediction)

  return float(error)

### Объединенная модель

In [6]:
from model import MolGen

In [ ]:
#gan_mol = pi.load(open('gan_mol.pkl', 'rb')) #загрузка предобученной на коформерах генеративной модели

In [ ]:
# gan_mol = pi.load(open('../weights/gan_mol7_50ksteps_100symb_kekuke.pkl', 'rb')) #загрузка предобученной только на ChemBL генеративной модели

In [8]:
gan_mol = torch.load('../weights/gan_mol7_50ksteps_100symb_kekuke.pkl', map_location=torch.device('cpu'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
#создание списка SMILES
data = []
with open('../data/database_cof_100smb_kekule.csv', "r") as f:
    for line in f.readlines()[1:]:
        data.append(line.split("\n")[0])

loader = gan_mol.create_dataloader(data, batch_size=20, shuffle=True, num_workers=10)

In [ ]:
# train model for 10000 steps
gan_mol.train()

In [ ]:
import warnings #возникает одна и та же ошибка
warnings.filterwarnings('ignore')

gan_mol.train_n_steps(loader, max_step=1000, evaluate_every=10)

### График измененеие clf loss в процессе обучения

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x1 = []
i = 0
for i in range (100):
  i += 1
  x1.append(i)

x2 = x1

In [ ]:
import math 

In [ ]:
# обучение предобученной на коформерах генеративной модели
y1 = []
y = [0.45, 0.5729166666666666, 0.52, 0.58, 0.6145833333333334, 0.57, 0.6224489795918368, 0.5729166666666666, 0.56, 0.62, 0.5208333333333334, 0.5918367346938775, 0.5416666666666666, 0.57, 0.5208333333333334, 0.6122448979591837, 0.5714285714285714, 0.59, 0.42857142857142855, 0.30612244897959184, 0.2916666666666667, 0.33, 0.46938775510204084, 0.36666666666666664, 0.6428571428571429, 0.75, 0.5, 0.5510204081632653, 0.44, 0.46875, 0.28125, 0.25510204081632654, 0.3333333333333333, 0.46875, 0.4479166666666667, 0.5222222222222223, 0.5638297872340425, 0.42857142857142855, 0.40816326530612246, 0.425531914893617, 0.4673913043478261, 0.5, 0.5520833333333334, 0.63, 0.5531914893617021, 0.5612244897959183, 0.46938775510204084, 0.4148936170212766, 0.4479166666666667, 0.3191489361702128, 0.4772727272727273, 0.3333333333333333, 0.40625, 0.46938775510204084, 0.5408163265306123, 0.5777777777777777, 0.6046511627906976, 0.4387755102040816, 0.336734693877551, 0.3020833333333333, 0.22448979591836735, 0.36, 0.4479166666666667, 0.4387755102040816, 0.44, 0.4787234042553192, 0.5612244897959183, 0.5638297872340425, 0.53125, 0.6111111111111112, 0.6, 0.7448979591836735, 0.7142857142857143, 0.7346938775510204, 0.7, 0.6938775510204082, 0.66, 0.4166666666666667, 0.4583333333333333, 0.4479166666666667, 0.4479166666666667, 0.46808510638297873, 0.5104166666666666, 0.5425531914893617, 0.5824175824175825, 0.6276595744680851, 0.5104166666666666, 0.47959183673469385, 0.44, 0.5217391304347826, 0.48, 0.3, 0.32608695652173914, 0.42857142857142855, 0.3125, 0.3020833333333333, 0.2765957446808511, 0.28125, 0.24489795918367346, 0.34]
for i in y:
  j = i ** (1/2)
  y1.append(j)

In [ ]:
# предобученной только на ChemBL генеративной модели
y2 = []
y = [0.6666666666666666, 0.618421052631579, 0.5135135135135135, 0.4050632911392405, 0.3, 0.34285714285714286, 0.2875, 0.23333333333333334, 0.4574468085106383, 0.5157894736842106, 0.7065217391304348, 0.5952380952380952, 0.5909090909090909, 0.5, 0.4878048780487805, 0.36764705882352944, 0.2830188679245283, 0.375, 0.44, 0.3220338983050847, 0.475, 0.7755102040816326, 0.7727272727272727, 0.66, 0.6428571428571429, 0.48214285714285715, 0.48333333333333334, 0.5625, 0.4069767441860465, 0.4090909090909091, 0.38372093023255816, 0.4880952380952381, 0.5116279069767442, 0.49333333333333335, 0.5945945945945946, 0.3977272727272727, 0.32222222222222224, 0.3333333333333333, 0.3522727272727273, 0.36046511627906974, 0.4431818181818182, 0.38095238095238093, 0.47674418604651164, 0.5851063829787234, 0.6222222222222222, 0.627906976744186, 0.7228915662650602, 0.6046511627906976, 0.6951219512195121, 0.7333333333333333, 0.6477272727272727, 0.7439024390243902, 0.5444444444444444, 0.7093023255813954, 0.5, 0.5, 0.44047619047619047, 0.5227272727272727, 0.40789473684210525, 0.34146341463414637, 0.45555555555555555, 0.4090909090909091, 0.4714285714285714, 0.48717948717948717, 0.5972222222222222, 0.4594594594594595, 0.5294117647058824, 0.5735294117647058, 0.5384615384615384, 0.47560975609756095, 0.4367816091954023, 0.5217391304347826, 0.4186046511627907, 0.36585365853658536, 0.4230769230769231, 0.46153846153846156, 0.47619047619047616, 0.4868421052631579, 0.5606060606060606, 0.5714285714285714, 0.4166666666666667, 0.5217391304347826, 0.38823529411764707, 0.3333333333333333, 0.4186046511627907, 0.43617021276595747, 0.36666666666666664, 0.40476190476190477, 0.45555555555555555, 0.5977011494252874, 0.46987951807228917, 0.4148936170212766, 0.4418604651162791, 0.36666666666666664, 0.3953488372093023, 0.38372093023255816, 0.5760869565217391, 0.47, 0.5416666666666666, 0.4791666666666667]
for i in y:
  j = i ** (1/2)
  y2.append(j)

In [ ]:
# построение графика
plt.figure(figsize=(15,10))
plt.plot(x1, y1, label = "Предобученная на коформерах модель")
plt.plot(x2, y2, label = "Предобученной только на ChemBL модель")
plt.title('Изменение clf loss в процессе обучения')
plt.xlabel('Итерация обучения')
plt.ylabel('clf loss mean')
plt.legend()
plt.show()